# 10. 데이터 집계와 그룹 연산

## 10.1 GroupBy 메카닉

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
df = pd.DataFrame({'key1': ['a','a','b','b','a'],
                  'key2': ['one','two','one','two','one'],
                  'data1': np.random.randn(5),
                  'data2': np.random.randn(5)})
df

,key1,key2,data1,data2
0,a,one,0.439959,2.093667
1,a,two,-1.393448,-1.485628
2,b,one,-0.359161,1.746042
3,b,two,-1.507964,-1.044350
4,a,one,-1.210993,-0.273857


In [4]:
grouped = df['data1'].groupby(df['key1'])
grouped

In [5]:
grouped.mean()

key1
a   -0.721494
b   -0.933562
Name: data1, dtype: float64

In [6]:
means = df['data1'].groupby([df['key1'], df['key2']]).mean()
means

key1  key2
a     one    -0.385517
      two    -1.393448
b     one    -0.359161
      two    -1.507964
Name: data1, dtype: float64

In [7]:
means.unstack()

key2,one,two
key1,,
a,-0.385517,-1.393448
b,-0.359161,-1.507964


In [8]:
states = np.array(['Ohio','California','California','Ohio','Ohio'])
years = np.array([2005,2005,2006,2005,2006])
df['data1'].groupby([states,years]).mean()

California  2005   -1.393448
            2006   -0.359161
Ohio        2005   -0.534003
            2006   -1.210993
Name: data1, dtype: float64

In [9]:
df.groupby('key1').mean()

,data1,data2
key1,,
a,-0.721494,0.111394
b,-0.933562,0.350846


In [10]:
df.groupby(['key1','key2']).mean()

data1     data2
key1 key2                    
a    one  -0.385517  0.909905
     two  -1.393448 -1.485628
b    one  -0.359161  1.746042
     two  -1.507964 -1.044350

In [11]:
df.groupby(['key1','key2']).size()

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

### 그룹 간 순회하기

In [12]:
for name, group in df.groupby('key1'):
    print(name)
    print(group)

a
  key1 key2     data1     data2
0    a  one  0.439959  2.093667
1    a  two -1.393448 -1.485628
4    a  one -1.210993 -0.273857
b
  key1 key2     data1     data2
2    b  one -0.359161  1.746042
3    b  two -1.507964 -1.044350


In [13]:
for (k1,k2), group in df.groupby(['key1','key2']):
    print((k1,k2))
    print(group)

('a', 'one')
  key1 key2     data1     data2
0    a  one  0.439959  2.093667
4    a  one -1.210993 -0.273857
('a', 'two')
  key1 key2     data1     data2
1    a  two -1.393448 -1.485628
('b', 'one')
  key1 key2     data1     data2
2    b  one -0.359161  1.746042
('b', 'two')
  key1 key2     data1    data2
3    b  two -1.507964 -1.04435


In [14]:
pieces = dict(list(df.groupby('key1')))
pieces['b']

,key1,key2,data1,data2
2,b,one,-0.359161,1.746042
3,b,two,-1.507964,-1.044350


In [16]:
df.dtypes

key1      object
key2      object
data1    float64
data2    float64
dtype: object

In [19]:
grouped = df.groupby(df.dtypes, axis=1)

In [20]:
for dtype, group in grouped:
    print(dtype)
    print(group)

float64
      data1     data2
0  0.439959  2.093667
1 -1.393448 -1.485628
2 -0.359161  1.746042
3 -1.507964 -1.044350
4 -1.210993 -0.273857
object
  key1 key2
0    a  one
1    a  two
2    b  one
3    b  two
4    a  one


### 컬럼이나 컬럼의 일부만 선택하기

In [23]:
df.groupby(['key1','key2'])[['data2']].mean()

data2
key1 key2          
a    one   0.909905
     two  -1.485628
b    one   1.746042
     two  -1.044350

In [24]:
s_grouped = df.groupby(['key1','key2'])['data2']
s_grouped

In [25]:
s_grouped.mean()

key1  key2
a     one     0.909905
      two    -1.485628
b     one     1.746042
      two    -1.044350
Name: data2, dtype: float64

### 사전과 Series에서 그룹핑하기

In [31]:
people=pd.DataFrame(np.random.randn(5,5),
                   columns=['a','b','c','d','e'],
                   index=['Joe','Steve','Wes','Jim','Travis'])
people

,a,b,c,d,e
Joe,-0.718943,-1.633547,-1.243321,-0.279232,1.213912
Steve,0.075323,-0.415829,1.283879,0.063221,-0.194291
Wes,-0.213722,-0.463340,1.311823,0.489126,-2.157077
Jim,0.496733,0.514411,1.161729,-2.032254,-0.852534
Travis,-0.388736,-1.911411,1.099107,0.330140,-1.206926


In [30]:
people.iloc[2:3, [1,2]]=np.nan
people

,a,b,c,d,e
Joe,-0.282620,-0.562649,-1.203131,-0.638500,1.255906
Steve,1.882804,0.040844,0.692652,0.275167,0.298909
Wes,-0.182683,NaN,NaN,1.529025,0.174676
Jim,1.260800,0.514284,-1.045299,-0.628619,-0.664595
Travis,0.595243,0.499269,2.303330,0.247539,-1.133911


In [36]:
mapping = {'a':'red', 'b':'red', 'c':'blue',
          'd': 'blue', 'e': 'red', 'f': 'orange'}

In [37]:
by_column = people.groupby(mapping, axis=1)
by_column.sum()

,blue,red
Joe,-1.522553,-1.138578
Steve,1.347100,-0.534798
Wes,0.489126,-2.370799
Jim,-0.870525,0.158610
Travis,1.429247,-3.507072


In [38]:
map_series = pd.Series(mapping)
map_series

a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

In [40]:
people.groupby(map_series, axis=1).count()

,blue,red
Joe,2,3
Steve,2,3
Wes,1,2
Jim,2,3
Travis,2,3


### 함수로 그룹핑하기

In [41]:
people.groupby(len).sum()

,a,b,c,d,e
3,-0.435932,-1.119136,-0.081592,-1.822361,-1.795699
5,0.075323,-0.415829,1.283879,0.063221,-0.194291
6,-0.388736,-1.911411,1.099107,0.330140,-1.206926


In [42]:
key_list =['one','one','one','two','two']

In [43]:
people.groupby([len, key_list]).min()

a         b         c         d         e
3 one -0.718943 -1.633547 -1.243321 -0.279232 -2.157077
  two  0.496733  0.514411  1.161729 -2.032254 -0.852534
5 one  0.075323 -0.415829  1.283879  0.063221 -0.194291
6 two -0.388736 -1.911411  1.099107  0.330140 -1.206926

### 색인 단계로 그룹핑하기

In [46]:
columns= pd.MultiIndex.from_arrays([['US','US','US','JP','JP'],
                                   [1,3,5,1,3]],
                                   names=['city','tenor'])
columns

MultiIndex([('US', 1),
            ('US', 3),
            ('US', 5),
            ('JP', 1),
            ('JP', 3)],
           names=['city', 'tenor'])

In [47]:
hier_df=pd.DataFrame(np.random.randn(4,5), columns=columns)
hier_df

city         US                            JP          
tenor         1         3         5         1         3
0     -0.369182  1.394089 -0.804548  0.113539  0.419044
1     -1.415726  0.078338 -0.158150  0.651352 -0.934537
2      0.176500 -0.112837 -0.162573 -0.104800  0.201741
3     -0.944723 -0.926800  0.342643  0.430347 -1.890207

In [49]:
hier_df.groupby(level='city',axis=1).count()

city,JP,US
0,2,3
1,2,3
2,2,3
3,2,3


## 10.2 데이터 집계

In [50]:
df

,key1,key2,data1,data2
0,a,one,0.439959,2.093667
1,a,two,-1.393448,-1.485628
2,b,one,-0.359161,1.746042
3,b,two,-1.507964,-1.044350
4,a,one,-1.210993,-0.273857


In [51]:
grouped = df.groupby('key1')

In [53]:
grouped['data1'].quantile(0.9)

key1
a    0.109768
b   -0.474041
Name: data1, dtype: float64

In [54]:
def peak_to_peak(arr):
    return arr.max() - arr.min()

In [55]:
grouped.agg(peak_to_peak)

,data1,data2
key1,,
a,1.833407,3.579295
b,1.148803,2.790392


In [56]:
grouped.describe()

data1                                                              \
     count      mean       std       min       25%       50%       75%   
key1                                                                     
a      3.0 -0.721494  1.009976 -1.393448 -1.302221 -1.210993 -0.385517   
b      2.0 -0.933562  0.812327 -1.507964 -1.220763 -0.933562 -0.646362   

               data2                                                    \
           max count      mean       std       min       25%       50%   
key1                                                                     
a     0.439959   3.0  0.111394  1.820481 -1.485628 -0.879743 -0.273857   
b    -0.359161   2.0  0.350846  1.973105 -1.044350 -0.346752  0.350846   

                          
           75%       max  
key1                      
a     0.909905  2.093667  
b     1.048444  1.746042

### 컬럼에 여러 가지 함수 적용하기

In [5]:
tips= pd.read_csv('C:/github/pydata-book-2nd-edition/examples/tips.csv')

In [6]:
tips['tip_pct']= tips['tip']/tips['total_bill']

In [61]:
tips[:6]

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808
5,25.29,4.71,No,Sun,Dinner,4,0.186240


In [62]:
grouped = tips.groupby(['day','smoker'])

In [63]:
grouped_pct = grouped['tip_pct']
grouped_pct.agg('mean')

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tip_pct, dtype: float64

In [65]:
grouped_pct.agg(['mean', 'std', peak_to_peak])

mean       std  peak_to_peak
day  smoker                                  
Fri  No      0.151650  0.028123      0.067349
     Yes     0.174783  0.051293      0.159925
Sat  No      0.158048  0.039767      0.235193
     Yes     0.147906  0.061375      0.290095
Sun  No      0.160113  0.042347      0.193226
     Yes     0.187250  0.154134      0.644685
Thur No      0.160298  0.038774      0.193350
     Yes     0.163863  0.039389      0.151240

In [66]:
grouped_pct.agg([('foo','mean'), ('bar', np.std)])

foo       bar
day  smoker                    
Fri  No      0.151650  0.028123
     Yes     0.174783  0.051293
Sat  No      0.158048  0.039767
     Yes     0.147906  0.061375
Sun  No      0.160113  0.042347
     Yes     0.187250  0.154134
Thur No      0.160298  0.038774
     Yes     0.163863  0.039389

In [67]:
functions = ['count', 'mean', 'max']

In [69]:
result= grouped['tip_pct', 'total_bill'].agg(functions)
result

<ipython-input-69-d2e5646cf0e4>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  result= grouped['tip_pct', 'total_bill'].agg(functions)


tip_pct                     total_bill                  
              count      mean       max      count       mean    max
day  smoker                                                         
Fri  No           4  0.151650  0.187735          4  18.420000  22.75
     Yes         15  0.174783  0.263480         15  16.813333  40.17
Sat  No          45  0.158048  0.291990         45  19.661778  48.33
     Yes         42  0.147906  0.325733         42  21.276667  50.81
Sun  No          57  0.160113  0.252672         57  20.506667  48.17
     Yes         19  0.187250  0.710345         19  24.120000  45.35
Thur No          45  0.160298  0.266312         45  17.113111  41.19
     Yes         17  0.163863  0.241255         17  19.190588  43.11

In [70]:
result['tip_pct']

count      mean       max
day  smoker                           
Fri  No          4  0.151650  0.187735
     Yes        15  0.174783  0.263480
Sat  No         45  0.158048  0.291990
     Yes        42  0.147906  0.325733
Sun  No         57  0.160113  0.252672
     Yes        19  0.187250  0.710345
Thur No         45  0.160298  0.266312
     Yes        17  0.163863  0.241255

In [76]:
ftuples = [('Durchschnitt', 'mean'), ('Abweichng', np.var)]

In [77]:
grouped['tip_pct', 'total_bill'].agg(ftuples)

<ipython-input-77-c2b2ffd79840>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  grouped['tip_pct', 'total_bill'].agg(ftuples)


tip_pct             total_bill            
            Durchschnitt Abweichng Durchschnitt   Abweichng
day  smoker                                                
Fri  No         0.151650  0.000791    18.420000   25.596333
     Yes        0.174783  0.002631    16.813333   82.562438
Sat  No         0.158048  0.001581    19.661778   79.908965
     Yes        0.147906  0.003767    21.276667  101.387535
Sun  No         0.160113  0.001793    20.506667   66.099980
     Yes        0.187250  0.023757    24.120000  109.046044
Thur No         0.160298  0.001503    17.113111   59.625081
     Yes        0.163863  0.001551    19.190588   69.808518

In [78]:
grouped.agg({'tip': np.max, 'size':'sum'})

tip  size
day  smoker             
Fri  No       3.50     9
     Yes      4.73    31
Sat  No       9.00   115
     Yes     10.00   104
Sun  No       6.00   167
     Yes      6.50    49
Thur No       6.70   112
     Yes      5.00    40

In [79]:
grouped.agg({'tip_pct': ['min','max','mean','std'],
            'size':'sum'})

tip_pct                               size
                  min       max      mean       std  sum
day  smoker                                             
Fri  No      0.120385  0.187735  0.151650  0.028123    9
     Yes     0.103555  0.263480  0.174783  0.051293   31
Sat  No      0.056797  0.291990  0.158048  0.039767  115
     Yes     0.035638  0.325733  0.147906  0.061375  104
Sun  No      0.059447  0.252672  0.160113  0.042347  167
     Yes     0.065660  0.710345  0.187250  0.154134   49
Thur No      0.072961  0.266312  0.160298  0.038774  112
     Yes     0.090014  0.241255  0.163863  0.039389   40

### 색인되지 않은 형태로 집계된 데이터 반환하기

In [82]:
tips.groupby(['day','smoker'], as_index=False).mean()

,day,smoker,total_bill,tip,size,tip_pct
0,Fri,No,18.420000,2.812500,2.250000,0.151650
1,Fri,Yes,16.813333,2.714000,2.066667,0.174783
2,Sat,No,19.661778,3.102889,2.555556,0.158048
3,Sat,Yes,21.276667,2.875476,2.476190,0.147906
4,Sun,No,20.506667,3.167895,2.929825,0.160113
5,Sun,Yes,24.120000,3.516842,2.578947,0.187250
6,Thur,No,17.113111,2.673778,2.488889,0.160298
7,Thur,Yes,19.190588,3.030000,2.352941,0.163863
